In [ ]:
pip install requests_cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
#librarys
import json
import requests
import numpy as np
import pandas as pd
import plotly.express as px
from requests_cache import CachedSession
from datetime import datetime

In [ ]:
api_key = '4a8b15e8da98bcfed048039b4851248d'

In [ ]:
def get_zip_weather(api_key,zipp, country = 'US'):
    """Get the weather for zip."""
    weather_url = f"https://api.openweathermap.org/data/2.5/weather?zip={zipp},{country}&appid={api_key}&units=imperial"
    response = requests.get(weather_url)
    return response.json()

In [ ]:
get_zip_weather(api_key,96729)

{'coord': {'lon': -157.0791, 'lat': 21.173},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 81.86,
  'feels_like': 85.15,
  'temp_min': 75.94,
  'temp_max': 81.86,
  'pressure': 1016,
  'humidity': 65},
 'visibility': 10000,
 'wind': {'speed': 8.05, 'deg': 20, 'gust': 17.27},
 'clouds': {'all': 40},
 'dt': 1702346658,
 'sys': {'type': 1,
  'id': 7870,
  'country': 'US',
  'sunrise': 1702313739,
  'sunset': 1702352865},
 'timezone': -36000,
 'id': 0,
 'name': 'Hoolehua',
 'cod': 200}

In [ ]:
def State_info(state):
    url = f'https://data.opendatasoft.com/api/explore/v2.1/catalog/datasets/georef-united-states-of-america-zc-point@public/records?limit=100&refine=stusps_code%3A%22{state}%22'
    session = CachedSession()
    response = session.get(url)
    return response.json()

In [ ]:
State_info('DE')

{'total_count': 67,
 'results': [{'zip_code': '19944',
   'usps_city': 'Fenwick Island',
   'stusps_code': 'DE',
   'ste_name': 'Delaware',
   'zcta': 'TRUE',
   'parent_zcta': None,
   'population': 697.0,
   'density': 212.2,
   'primary_coty_code': '10005',
   'primary_coty_name': 'Sussex',
   'county_weights': '{"10005": "100"}',
   'coty_name': ['Sussex'],
   'cty_code': ['10005'],
   'imprecise': 'FALSE',
   'military': 'FALSE',
   'timezone': 'America/New_York',
   'geo_point_2d': {'lon': -75.05559, 'lat': 38.47578}},
  {'zip_code': '19709',
   'usps_city': 'Middletown',
   'stusps_code': 'DE',
   'ste_name': 'Delaware',
   'zcta': 'TRUE',
   'parent_zcta': None,
   'population': 42480.0,
   'density': 205.9,
   'primary_coty_code': '10003',
   'primary_coty_name': 'New Castle',
   'county_weights': '{"10003": "100"}',
   'coty_name': ['New Castle'],
   'cty_code': ['10003'],
   'imprecise': 'FALSE',
   'military': 'FALSE',
   'timezone': 'America/New_York',
   'geo_point_2d': {

### Current Weather

In [ ]:
def get_weather_map():
    api_key = '4a8b15e8da98bcfed048039b4851248d'
    try:
        state = str(input("Please enter the USPS state code (ie. California is CA): "))
        stateinfo = State_info(state)
        dfstateinfo = pd.DataFrame(stateinfo['results'])
    except ValueError:
        print("Invalid input. Please enter a valid state name. It is cap sensitive.")
    if stateinfo['total_count'] == 0:
        print("Invalid input. Please enter a valid state name. It is cap sensitive.")
    # county info
    print(set(dfstateinfo['primary_coty_name']))
    county = str(input(f'Please choose the county in {state} you would like to see from the list above:'))
    zipcodes = dfstateinfo['zip_code'][dfstateinfo['primary_coty_name'] == county]

    combined = []
    for zipcode in zipcodes:
        test_data = get_zip_weather(api_key,zipcode)
        #gather subseted data
        weather_info = test_data['weather'][0]
        main_info = test_data['main']
        wind_info = test_data['wind']
        loc_info = test_data['coord']
        city_info = test_data['name']
        # join all to a dictionary
        sub = {**{'name':city_info},
               **{'zip':zipcode},
               **weather_info,
               **main_info,
               **wind_info,
               **loc_info}
        combined.append(sub)
    df = pd.DataFrame(combined)
    df.head()


    fig = px.scatter_mapbox(df,
                        lon = df['lon'],
                        lat = df['lat'],
                        hover_name = df['name'],
                        hover_data = df[['feels_like','temp_min','temp_max','humidity']],
                        zoom = 5,
                        color = df['temp'],
                        width = 900,
                        height = 600,
                        size = df['speed'],
                        color_continuous_scale="temps",
                        title = f'Weather Map of {county} county')
    fig.update_layout(mapbox_style = 'carto-positron')
    fig.show()

In [ ]:
# get_weather_map()

In [ ]:
# state = str(input("Please enter the USPS state code (ie. California is CA): "))
# stateinfo = State_info(state)
# dfstateinfo = pd.DataFrame(stateinfo['results'])

In [ ]:
# dfstateinfo

In [ ]:
# zipcodes = dfstateinfo['zip_code'][dfstateinfo['primary_coty_name'] == 'Los Angeles']
zipcodes = [95616]
combined = []
for zipcode in zipcodes:
    test_data = get_zip_weather(api_key,zipcode)
    #gather subseted data
    weather_info = test_data['weather'][0]
    main_info = test_data['main']
    wind_info = test_data['wind']
    loc_info = test_data['coord']
    city_info = test_data['name']
    # join all to a dictionary
    sub = {**{'name':city_info},
            **{'zip':zipcode},
            **weather_info,
            **main_info,
            **wind_info,
            **loc_info}
    combined.append(sub)
df = pd.DataFrame(combined)
df.head()

,name,zip,id,main,description,icon,temp,feels_like,temp_min,temp_max,pressure,humidity,speed,deg,lon,lat
0,Davis,95616,800,Clear,clear sky,01n,51.04,48.96,46.17,54.79,1016,66,0,0,-121.7418,38.5538


### Historical and Forecast

In [ ]:
# https://history.openweathermap.org/data/2.5/history/city?q=London,UK&appid={API key}

#
def get_hist_weather(api_key, city, country = 'US'):
    """Get 24 hour historical weather for city name."""
    weather_url = f"https://history.openweathermap.org/data/2.5/history/city?q={city},{country}&cnt={24}&appid={api_key}&units=imperial"
    response = requests.get(weather_url)
    return response.json()

def get_forecast_weather(api_key, city, country = 'US'):
    """Get 4 day hourly forecast for city name."""
    weather_url = f"https://pro.openweathermap.org/data/2.5/forecast/hourly?q={city},{country}&appid={api_key}&units=imperial"
    response = requests.get(weather_url)
    return response.json()

In [ ]:
hist_data = get_hist_weather(api_key, 'Davis')

forecast_data = get_forecast_weather(api_key, 'Davis')

In [ ]:
hist_data

{'message': 'Count: 24',
 'cod': '200',
 'city_id': 5341704,
 'calctime': 0.025661462,
 'cnt': 24,
 'list': [{'dt': 1702256400,
   'main': {'temp': 52.02,
    'feels_like': 49.6,
    'pressure': 1024,
    'humidity': 57,
    'temp_min': 49.89,
    'temp_max': 54.32},
   'wind': {'speed': 0, 'deg': 0},
   'clouds': {'all': 0},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01n'}]},
  {'dt': 1702260000,
   'main': {'temp': 48.25,
    'feels_like': 48.25,
    'pressure': 1023,
    'humidity': 67,
    'temp_min': 45.14,
    'temp_max': 50.76},
   'wind': {'speed': 0, 'deg': 0},
   'clouds': {'all': 0},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01n'}]},
  {'dt': 1702263600,
   'main': {'temp': 46.06,
    'feels_like': 46.06,
    'pressure': 1023,
    'humidity': 71,
    'temp_min': 44.15,
    'temp_max': 48.97},
   'wind': {'speed': 0, 'deg': 0},
   'clouds': {'all': 0},
   'weather': [{'i

In [ ]:
# Build historical data frame
date_time = pd.DataFrame([pd.to_datetime(hist_data['list'][i]['dt'], unit='s') for i in range(len(hist_data['list']))])
main_data = pd.DataFrame([hist_data['list'][i]['main'] for i in range(len(hist_data['list']))])
wind_data = pd.DataFrame([hist_data['list'][i]['wind'] for i in range(len(hist_data['list']))])
cloud_data = pd.DataFrame([hist_data['list'][i]['clouds'] for i in range(len(hist_data['list']))])
weather_data = pd.DataFrame([hist_data['list'][i]['weather'][0] for i in range(len(hist_data['list']))])

hist_df = pd.concat([date_time, main_data, wind_data, cloud_data, weather_data], axis=1)
hist_df['data_type'] = "Historical"
hist_df

,0,temp,feels_like,pressure,humidity,temp_min,temp_max,speed,deg,all,id,main,description,icon,data_type
0,2023-12-11 01:00:00,52.02,49.60,1024,57,49.89,54.32,0,0,0,800,Clear,clear sky,01n,Historical
1,2023-12-11 02:00:00,48.25,48.25,1023,67,45.14,50.76,0,0,0,800,Clear,clear sky,01n,Historical
2,2023-12-11 03:00:00,46.06,46.06,1023,71,44.15,48.97,0,0,0,800,Clear,clear sky,01n,Historical
3,2023-12-11 04:00:00,45.59,45.59,1023,69,42.69,47.77,0,0,0,800,Clear,clear sky,01n,Historical
4,2023-12-11 05:00:00,42.66,42.66,1023,80,39.20,46.76,0,0,0,800,Clear,clear sky,01n,Historical
5,2023-12-11 06:00:00,40.46,40.46,1023,86,37.40,43.99,0,0,0,800,Clear,clear sky,01n,Historical
6,2023-12-11 07:00:00,39.54,39.54,1022,90,37.29,42.98,0,0,0,800,Clear,clear sky,01n,Historical
7,2023-12-11 08:00:00,37.96,37.96,1022,87,33.80,41.79,0,0,0,800,Clear,clear sky,01n,Historical
8,2023-12-11 09:00:00,36.82,36.82,1021,89,33.80,40.80,0,0,0,800,Clear,clear sky,01n,Historical
9,2023-12-11 10:00:00,35.89,35.89,1021,93,32.11,40.80,0,0,0,800,Clear,clear sky,01n,Historical


In [ ]:
forecast_data

{'cod': '200',
 'message': 0,
 'cnt': 96,
 'list': [{'dt': 1702350000,
   'main': {'temp': 34,
    'feels_like': 34,
    'temp_min': 34,
    'temp_max': 34.93,
    'pressure': 1018,
    'sea_level': 1018,
    'grnd_level': 873,
    'humidity': 75,
    'temp_kf': -0.52},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02n'}],
   'clouds': {'all': 19},
   'wind': {'speed': 2.73, 'deg': 82, 'gust': 3.15},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'n'},
   'dt_txt': '2023-12-12 03:00:00'},
  {'dt': 1702353600,
   'main': {'temp': 34.09,
    'feels_like': 31.39,
    'temp_min': 34.09,
    'temp_max': 34.56,
    'pressure': 1019,
    'sea_level': 1019,
    'grnd_level': 873,
    'humidity': 73,
    'temp_kf': -0.26},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02n'}],
   'clouds': {'all': 20},
   'wind': {'speed': 3.22, 'deg': 88, 'gust': 3.38},
   'visibility': 10000,
   'pop

In [ ]:
# Build forecast data frame
date_time = pd.DataFrame([pd.to_datetime(forecast_data['list'][i]['dt'], unit='s') for i in range(len(forecast_data['list']))])
main_data = pd.DataFrame([forecast_data['list'][i]['main'] for i in range(len(forecast_data['list']))])
wind_data = pd.DataFrame([forecast_data['list'][i]['wind'] for i in range(len(forecast_data['list']))])
cloud_data = pd.DataFrame([forecast_data['list'][i]['clouds'] for i in range(len(forecast_data['list']))])
weather_data = pd.DataFrame([forecast_data['list'][i]['weather'][0] for i in range(len(forecast_data['list']))])


forecast_df = pd.concat([date_time, main_data, wind_data, cloud_data, weather_data], axis=1)
forecast_df['data_type'] = "Forecast"
forecast_df

,0,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,temp_kf,speed,deg,gust,all,id,main,description,icon,data_type
0,2023-12-12 03:00:00,34.00,34.00,34.00,34.93,1018,1018,873,75,-0.52,2.73,82,3.15,19,801,Clouds,few clouds,02n,Forecast
1,2023-12-12 04:00:00,34.09,31.39,34.09,34.56,1019,1019,873,73,-0.26,3.22,88,3.38,20,801,Clouds,few clouds,02n,Forecast
2,2023-12-12 05:00:00,34.11,31.01,34.11,34.32,1019,1019,872,71,-0.12,3.56,90,3.38,25,802,Clouds,scattered clouds,03n,Forecast
3,2023-12-12 06:00:00,34.20,31.03,34.20,34.30,1020,1020,873,69,-0.06,3.62,75,3.15,37,802,Clouds,scattered clouds,03n,Forecast
4,2023-12-12 07:00:00,34.66,31.39,34.66,34.66,1021,1021,872,66,0.00,3.78,84,2.98,100,804,Clouds,overcast clouds,04n,Forecast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2023-12-15 22:00:00,43.81,43.81,43.81,43.81,1030,1030,883,49,0.00,1.63,25,2.71,31,802,Clouds,scattered clouds,03d,Forecast
92,2023-12-15 23:00:00,42.49,42.49,42.49,42.49,1030,1030,883,55,0.00,2.75,29,3.38,41,802,Clouds,scattered clouds,03d,Forecast
93,2023-12-16 00:00:00,40.06,38.44,40.06,40.06,1031,1031,883,59,0.00,3.00,58,3.38,48,802,Clouds,scattered clouds,03d,Forecast
94,2023-12-16 01:00:00,38.61,35.53,38.61,38.61,1032,1032,883,60,0.00,4.18,77,3.60,42,802,Clouds,scattered clouds,03n,Forecast


In [ ]:
#merge data for plotting
plot_df = pd.concat([hist_df, forecast_df], ignore_index=True, sort=True, axis=0)

In [ ]:
plot_df

,0,all,data_type,deg,description,feels_like,grnd_level,gust,humidity,icon,id,main,pressure,sea_level,speed,temp,temp_kf,temp_max,temp_min
0,2023-12-11 01:00:00,0,Historical,0,clear sky,49.60,NaN,NaN,57,01n,800,Clear,1024,NaN,0.00,52.02,NaN,54.32,49.89
1,2023-12-11 02:00:00,0,Historical,0,clear sky,48.25,NaN,NaN,67,01n,800,Clear,1023,NaN,0.00,48.25,NaN,50.76,45.14
2,2023-12-11 03:00:00,0,Historical,0,clear sky,46.06,NaN,NaN,71,01n,800,Clear,1023,NaN,0.00,46.06,NaN,48.97,44.15
3,2023-12-11 04:00:00,0,Historical,0,clear sky,45.59,NaN,NaN,69,01n,800,Clear,1023,NaN,0.00,45.59,NaN,47.77,42.69
4,2023-12-11 05:00:00,0,Historical,0,clear sky,42.66,NaN,NaN,80,01n,800,Clear,1023,NaN,0.00,42.66,NaN,46.76,39.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2023-12-15 22:00:00,31,Forecast,25,scattered clouds,43.81,883.0,2.71,49,03d,802,Clouds,1030,1030.0,1.63,43.81,0.0,43.81,43.81
116,2023-12-15 23:00:00,41,Forecast,29,scattered clouds,42.49,883.0,3.38,55,03d,802,Clouds,1030,1030.0,2.75,42.49,0.0,42.49,42.49
117,2023-12-16 00:00:00,48,Forecast,58,scattered clouds,38.44,883.0,3.38,59,03d,802,Clouds,1031,1031.0,3.00,40.06,0.0,40.06,40.06
118,2023-12-16 01:00:00,42,Forecast,77,scattered clouds,35.53,883.0,3.60,60,03n,802,Clouds,1032,1032.0,4.18,38.61,0.0,38.61,38.61


In [ ]:

fig = px.line(plot_df, x=0, y="temp", color="data_type")
# fig.add_scatter(plot_df, x=0, y="temp_min", color="data_type")

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_df[0], y=plot_df["temp"],
                    line_shape='linear'))

In [ ]:
#TODO: Plot historical and forecast data on same plot. Include lines for temp, max temp, min temp. Include hover option for other measures like wind speed, weather description, feels like.


# Create the plot
fig = px.line(plot_df, x=0, y=['temp', 'temp_max', 'temp_min'],
              labels={'value': 'Temperature (°C)', 'variable': 'Temperature Type'},
              title='Historical and Forecast Temperature Data')

# Add hover data
fig.update_traces(mode='lines+markers')
for trace in fig.data:
    trace.name = trace.name.replace('_',' ').title()
fig.update_layout(hovermode='x unified')
fig.add_trace(
    px.scatter(plot_df, x=0, y='temp', custom_data=['speed', 'description']).data[0]
)
fig.update_traces(hovertemplate="<br>".join([
    "Date: %{x}",
    "Temperature: %{y}°C",
    "Wind Speed: %{customdata[0]} km/h",
    "Weather: %{customdata[1]}",
]))

# Show the plot
fig.show()